<a href="https://colab.research.google.com/github/david-j-cox/Man-vs-Machine/blob/master/add_so_col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
# Set working directory
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/GME & MLB/Scripts and Notebooks/'

Mounted at /content/gdrive
/content/gdrive/My Drive/GME & MLB/Scripts and Notebooks


In [ ]:
# Connect GPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [2]:
# Connect TPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.3.190.202:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.3.190.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.3.190.202:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
# Packages and modules we'll use
# System
import glob
from google.colab import files
import zipfile, io, os
import warnings
import time

# Data manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display, clear_output

print("Packages loaded")

Packages loaded


### Functions

In [12]:
temp_df.description[52372]

'ball'

In [13]:
# Function to get change in so pre-post pitch
def so_change(df, year):
  """
  Add single column with the change in strike and out value resulting each pitch in the dataset.
    1.) Change in strikes and outs (so) on a Continuous scale. 
  -----
  Params:
    df: dataframe
      - Dataframe containing the data with the strike and out states and cols necessary 
        to temporally sort the data. 
    year: int
      - The year within the dataframe you want to isolate for batch computing. 
  -----
  Returns: 
    The original dataframe with an added column containing the mapped so value for each pitch. 
  """

  # Create list of all Strike and out categories to serve as reinforcers
  s_o = ['hit_into_play', 'foul','swinging_strike', 'swinging_strike_blocked', \
        'called_strike','foul_bunt', 'foul_tip', 'missed_bunt','swinging_pitchout', \
        'foul_pitchout', 'bunt_foul_tip', 'unknown_strike']

  df = df.sort_values(by=['player_name', 'game_date', 'game_pk', 'inning', 'outs_when_up', 'pitch_number'])
  df = df.reset_index(drop=True)

  # CHANGE IN SO
  temp_list = []  # Empty list to store change values
  for index in range(len(df)):
    # First pitch in the dataframe
    val = df['description'][index]
    if val in s_o:
      temp_list.append(1)
    else:
      temp_list.append(0)
    
    # Providing notebook user with updating progress after every 100th loop. 
    if index%100==0:
      clear_output()
      print(f'Season: {year}\nSO Change: {index} of {len(df)} completed ({int((index/len(df))*100)}%)')

  # Add data to the original passed dataframe
  df['so_change'] = temp_list

  return df

# Add changes in RE88

In [14]:
# Add RE88 changes to all dfs
for year in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
  temp_df = pd.read_csv(f'../Data/02_intermediate/df_{year}.csv')
  drop_list = [i for i in list(temp_df) if 'Unnamed' in i]
  temp_df = temp_df.drop(drop_list, axis=1)
  temp_df = temp_df.reset_index(drop=True)
  for col in ['on_1b', 'on_2b', 'on_3b']:
    temp_df[col] = temp_df[col].fillna(0)
  temp_df = so_change(temp_df, year)
  temp_df.to_csv(f'../Data/02_intermediate/df_{year}.csv')

# Take a look at the result
temp_df.head(20)

Season: 2019
SO Change: 732400 of 732473 completed (99%)


,game_year,game_pk,pitch_type,game_date,type,balls,strikes,inning_topbot,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,events,description,inning,pitcher,player_name,batter,on_1b,on_2b,on_3b,outs_when_up,re_24,re_24_change_raw,re_24_change_ord,so_change
0,2019,566106,FF,2019-05-11,B,0,0,Bot,1,4-Seam Fastball,2,1,2,1,NaN,ball,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,0
1,2019,566106,FF,2019-05-11,S,1,0,Bot,2,4-Seam Fastball,2,1,2,1,NaN,called_strike,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1
2,2019,566106,SL,2019-05-11,S,1,1,Bot,3,Slider,2,1,2,1,NaN,foul,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1
3,2019,566106,SL,2019-05-11,S,1,2,Bot,4,Slider,2,1,2,1,NaN,foul,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1
4,2019,566106,SL,2019-05-11,S,1,2,Bot,5,Slider,2,1,2,1,NaN,foul,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1
5,2019,566106,SL,2019-05-11,X,1,2,Bot,6,Slider,2,1,2,1,field_out,hit_into_play,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1
6,2019,566106,SL,2019-05-11,S,0,0,Bot,1,Slider,2,1,2,1,NaN,swinging_strike,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.218,1,1
7,2019,566106,CH,2019-05-11,B,0,1,Bot,2,Changeup,2,1,2,1,NaN,ball,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.000,0,0
8,2019,566106,CH,2019-05-11,B,1,1,Bot,3,Changeup,2,1,2,1,NaN,ball,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.000,0,0
9,2019,566106,SL,2019-05-11,S,2,1,Bot,4,Slider,2,1,2,1,NaN,swinging_strike,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.000,0,1
